Instalacja biblioteki kaggle umożliwiającej pobranie datasetu

In [1]:
! pip install -q kaggle

In [131]:
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import niezbędnych bibliotek

In [240]:
import kaggle
import os
import pandas as pd
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

Ustawienie credentials i pobranie datasetu

In [ ]:
! mkdir ~/.kaggle 
! cp kaggle.json ~/.kaggle/ 
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
! mkdir reviews
! unzip imdb-dataset-of-50k-movie-reviews.zip -d reviews

Zapisanie datasetu jako dataframe'u pandasa

In [225]:
df = pd.read_csv('reviews/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Szczegóły analizowanego dataframe'u

In [226]:
print('Length of particular sentiments: \n{}\n'.format(df.groupby(['sentiment'])['sentiment'].count()))
print('Number of nulls: \n{}\n'.format(df.isnull().sum()))
print('Dataset info:')
df.info()

Length of particular sentiments: 
sentiment
negative    25000
positive    25000
Name: sentiment, dtype: int64

Number of nulls: 
review       0
sentiment    0
dtype: int64

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


Zamiana wydźwięku recenzji z positive/negative na maszynowe zera i jednyki

In [227]:
df['sentiment'] = df['sentiment'].replace('positive', 1)
df['sentiment'] = df['sentiment'].replace('negative', 0)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


Pozbycie się z treści recenzji słów przestankowych, specjalnych, liczb i innych. Rozszerzenie listy usuwanych słów o 'br'

In [228]:
X = df['review'].copy()
y = df['sentiment'].copy()

core = []

ps = PorterStemmer()

nltk.download('stopwords')
new_stopwords = ["br"]
stpwrd = nltk.corpus.stopwords.words('english')
stpwrd.extend(new_stopwords)

# def core(x):
#   x = re.sub('[^a-zA-Z]', ' ', x)
#   x = x.lower()
#   x = x.split()
#   x = [ps.stem(word) for word in x  if word not in set(stpwrd)]
#   return x

# HYBRYDA OBU???

# df['review'] = df['review'].apply(core)

for i in range(len(X)):
  x = re.sub('[^a-zA-Z]', ' ', X[i])
  x = x.lower()
  x = x.split()
  x = [ps.stem(word) for word in x  if word not in set(stpwrd)]
  x = " ".join(x)
  core.append(x)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [229]:
cv = TfidfVectorizer(max_features=5000)
X = cv.fit_transform(core).toarray()

In [236]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=98
)

print('X_train: {}\nX_test: {}\ny_train: {}\ny_test: {}'.format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))

X_train: (40000, 5000)
X_test: (10000, 5000)
y_train: (40000,)
y_test: (10000,)


In [235]:
# zmienić default?
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB()

In [238]:
prediction = mnb.predict(X_test)

In [241]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.85      0.84      0.84      4987
           1       0.84      0.86      0.85      5013

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



In [263]:
y_test

38454    0
1478     0
47941    0
22150    1
25178    1
        ..
43971    0
27154    0
1161     1
39756    0
6925     1
Name: sentiment, Length: 10000, dtype: int64

In [271]:
# prediction = pd.Series(prediction)
df_compare = pd.DataFrame(zip(y_test, prediction), columns=['true_value', 'prediction'])
df_compare.query('true_value != prediction')

TypeError: ignored

In [278]:
aaaa = pd.DataFrame(zip(y_test, prediction), columns=['true_value', 'prediction'])

In [287]:
bbbb = aaaa.query('true_value != prediction')
bbbb

,true_value,prediction
3,1,0
5,0,1
8,0,1
12,1,0
40,1,0
...,...,...
9987,0,1
9989,1,0
9991,1,0
9992,0,1
